# **Importing the libraries**

In [45]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import Sequential
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.layers import Dense
from keras.preprocessing import image
import pandas as pd
import os
import numpy as np
from PIL.Image import open

# **Reading the dataset**

In [46]:
brain_df = pd.read_csv('/content/drive/MyDrive/Brain Tumor.csv',usecols=[0,1])
brain_df.head(5)

,Image,Class
0,Image1,0
1,Image2,0
2,Image3,1
3,Image4,1
4,Image5,0


In [47]:
#storing the path of all the images
path_list = []
base_path = '/content/drive/MyDrive/Brain Tumor/'
for entry in os.listdir(base_path):
    path_list.append( os.path.join(base_path,entry))
pathes_dict={os.path.splitext(os.path.basename(x))[0]: x for x in path_list}
brain_df['path'] = brain_df['Image'].map(pathes_dict.get)

In [48]:
#pixel array for all the images
brain_df['pixels']=brain_df['path'].map(lambda x:np.asarray(open(x).resize((224,224))))

In [49]:
image_list = []
for i in range(0,len(brain_df)):
    brain_img = brain_df['pixels'][i].astype(np.float32)
    img_array = image.img_to_array(brain_img)
    image_list.append(keras.applications.mobilenet_v2.preprocess_input(img_array))
X = np.array(image_list)

In [50]:
y = np.array(brain_df.Class)

# **Splitting the dataset into training and testing**

In [51]:
#Ratio of split is 80:20
si = int(0.8*len(brain_df))
X_train = X[:si]
X_test = X[si:]
y_train = y[:si]
y_test = y[si:]

# **Building the model**

In [52]:
num_classes = 1
model = Sequential()
model.add(MobileNet(input_shape=(224, 224, 3),weights="imagenet",include_top=False))
model.add(keras.layers.GlobalAveragePooling2D())
model.add( Dense(num_classes, activation='sigmoid',name='preds'))
model.layers[0].trainable= False
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 preds (Dense)               (None, 1)                 1025      
                                                                 
Total params: 3,229,889
Trainable params: 1,025
Non-trainable params: 3,228,864
_________________________________________________________________


In [53]:
model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.SGD(lr=0.001),
    metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [54]:
model.fit(
    X_train[:,:,:,:],
    y_train[:],
    epochs=25,
    verbose=1,
    validation_data=(X_test[:,:,:,:], y_test[:])
)

Epoch 1/25
95/95 [==============================] - 130s 1s/step - loss: 0.7070 - accuracy: 0.5567 - val_loss: 0.6877 - val_accuracy: 0.5697
Epoch 2/25
95/95 [==============================] - 127s 1s/step - loss: 0.5576 - accuracy: 0.7132 - val_loss: 0.6394 - val_accuracy: 0.6401
Epoch 3/25
95/95 [==============================] - 127s 1s/step - loss: 0.4904 - accuracy: 0.7853 - val_loss: 0.5491 - val_accuracy: 0.7264
Epoch 4/25
95/95 [==============================] - 126s 1s/step - loss: 0.4517 - accuracy: 0.8102 - val_loss: 0.5223 - val_accuracy: 0.7570
Epoch 5/25
95/95 [==============================] - 127s 1s/step - loss: 0.4261 - accuracy: 0.8298 - val_loss: 0.4439 - val_accuracy: 0.8021
Epoch 6/25
95/95 [==============================] - 127s 1s/step - loss: 0.4087 - accuracy: 0.8368 - val_loss: 0.4765 - val_accuracy: 0.7915
Epoch 7/25
95/95 [==============================] - 129s 1s/step - loss: 0.3917 - accuracy: 0.8455 - val_loss: 0.5028 - val_accuracy: 0.7716
Epoch 8/25
95

In [55]:
model.save("model_brain.h5")

In [56]:
pretrained_cnn = keras.models.load_model('./model_brain.h5')
eval_score = pretrained_cnn.evaluate(X_test,y_test)
print('Eval loss:',eval_score[0])
print('Eval accuracy:',eval_score[1])

24/24 [==============================] - 27s 1s/step - loss: 0.3334 - accuracy: 0.8712
Eval loss: 0.33343520760536194
Eval accuracy: 0.8711819648742676


In [57]:
y_pred = (pretrained_cnn.predict(X_test) > 0.5)*1

# **Predicting for a testing image**

In [58]:
test_image = image.load_img("/content/drive/MyDrive/Brain Tumor/testing image/testimage1.jpg",target_size=(224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)
if(result[0][0]>=0.5):
  print("Tumor")
else:
  print("Non Tumor")

Non Tumor
